Install Libraries

In [ ]:
! pip install langchain langchain-community openai groq tiktoken pinecone-client langchain_pinecone unstructured pdfminer==20191125 pdfminer.six==20221105 pillow_heif unstructured_inference sentence-transformers

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, WebBaseLoader, YoutubeLoader, DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
from openai import OpenAI
import numpy as np
import tiktoken
import os
from groq import Groq

Initialize HuggingFace Embeddings client

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

<ipython-input-3-3a8dd36e4865>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

In [ ]:
text = "Hello my name is Saksham"

query_result = embeddings.embed_query(text)

In [ ]:
len(query_result)

768

# Calculating sentence similarity with embeddings (cosine-similarity)

In [ ]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
    model = SentenceTransformer(model_name)
    return model.encode(text)


def cosine_similarity_between_sentences(sentence1, sentence2):
    # Get embeddings for both sentences
    embedding1 = np.array(get_huggingface_embeddings(sentence1))
    embedding2 = np.array(get_huggingface_embeddings(sentence2))

    # Reshape embeddings for cosine_similarity function
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    print("Embedding for Sentence 1:", embedding1)
    print("\nEmbedding for Sentence 2:", embedding2)

    # Calculate cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)
    return similarity[0][0]


# Example usage
sentence1 = "I like walking to the park"
sentence2 = "I like walking to the office"


similarity = cosine_similarity_between_sentences(sentence1, sentence2)
print(f"\n\nCosine similarity between '{sentence1}' and '{sentence2}': {similarity:.4f}")

Embedding for Sentence 1: [[-5.18317223e-02  5.11822924e-02  1.72791979e-03 -1.36199202e-02
  -1.06869487e-03  2.96393428e-02 -4.72495109e-02 -2.11421009e-02
   5.48422784e-02  2.37766728e-02 -8.88856407e-03  1.03983462e-01
   1.87567491e-02 -6.70846319e-03 -3.84319052e-02 -7.80754834e-02
  -5.44624683e-03  6.69373479e-03 -1.80737115e-02  3.50141115e-02
  -3.07590049e-02  3.44667286e-02 -5.48802782e-03 -2.29204204e-02
   9.91370343e-03 -1.50746480e-02  1.37100741e-02 -3.11791096e-02
   7.79691711e-02  3.52224708e-02 -1.94613449e-02 -1.78903583e-02
   2.13377569e-02 -1.85624994e-02  1.29274099e-06  7.14496849e-03
  -7.68434315e-04  1.04230363e-02  3.67814861e-02 -3.46986540e-02
   3.50453630e-02  1.30667230e-02  1.00722872e-02 -4.18642862e-03
   2.04598345e-02 -2.74207480e-02  3.01958937e-02  2.14188918e-02
  -6.43193796e-02  1.04757305e-02 -4.66440478e-03 -4.05048616e-02
  -5.80140166e-02  1.99005734e-02 -2.49033840e-03  8.85135308e-02
   6.04227521e-02  1.96583439e-02  5.06717786e-02 

In [ ]:
print(len(get_huggingface_embeddings(sentence1)))

768


# Load the data

In [ ]:
! kaggle datasets download -d ayoubcherguelaine/company-documents-dataset
! unzip company-documents-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/ayoubcherguelaine/company-documents-dataset
License(s): apache-2.0
company-documents-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  company-documents-dataset.zip
replace CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_1.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_1.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_2.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_3.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_4.pdf  
  inflating: CompanyDocuments/Inventory Report/monthly-Category/monthly-Category/StockReport_2016-07_5.pdf  
  inflating: CompanyDocuments/Inventory Report/

In [ ]:
def process_directory(directory_path):
    data = []
    for root, _, files in os.walk(directory_path):
        for file in files:

            file_path = os.path.join(root, file)
            print(f"Processing file: {file_path}")
            loader = PyPDFLoader(file_path)
            data.append({"File": file_path, "Data": loader.load()})

    return data

directory_path = "/content/CompanyDocuments"
documents = process_directory(directory_path)


Processing file: /content/CompanyDocuments/invoices/invoice_10402.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_11067.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10515.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10651.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10339.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10624.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10919.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10607.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_11012.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_11009.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10878.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10666.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10992.pdf
Processing file: /content/CompanyDocuments/invoices/invoice_10319.pdf
Processing file: /co

# Set up Pinecone

In [ ]:
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

index_name = "rag-workshop"

namespace = "company-documents"

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

# Insert data into Pinecone

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
import os

pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

index_name = "rag-workshop"
namespace = "company-documents"

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings, namespace=namespace)

In [ ]:
import multiprocessing

def process_directory(directory_path):
       data = []
       for root, _, files in os.walk(directory_path):
           for file in files:
               file_path = os.path.join(root, file)
               loader = PyPDFLoader(file_path)
               loaded_documents = loader.load()  # Load documents using the loader
               for doc in loaded_documents:
                   # Assuming 'page_content' is the key for content in loaded documents
                   doc.metadata['source'] = file_path
                   data.append(doc)

       return data

def add_documents_batch(batch):
    """Adds a batch of documents to the Pinecone index."""
    vectorstore.add_documents(batch)

def process_directory_parallel(directory_path, num_processes=None):
    """Processes a directory of PDF files and adds them to Pinecone in parallel."""
    data = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            loader = PyPDFLoader(file_path)
            loaded_documents = loader.load()  # Load documents using the loader
            for doc in loaded_documents:
                # Assuming 'page_content' is the key for content in loaded documents
                doc.metadata['source'] = file_path
                data.append(doc)

    if num_processes is None:
        num_processes = multiprocessing.cpu_count()  # Use all available cores

    batch_size = max(1, len(data) // num_processes)
    batches = [data[i : i + batch_size] for i in range(0, len(data), batch_size)]


    # with multiprocessing.Pool(processes=num_processes) as pool:
    #     pool.map(add_documents_batch, batches)
    for batch in batches:
        add_documents_batch(batch)

# Usage
directory_path = "/content/CompanyDocuments"
process_directory_parallel(directory_path)

# Batch processing
# Assuming 'pinecone_index' is already defined
BATCH_SIZE = 1  # Experiment with batch sizes like 16, 32, or 64
for i in range(0, len(documents), BATCH_SIZE):
    batch = documents[i : i + BATCH_SIZE]
    print(f"Inserting batch {i // BATCH_SIZE + 1} of {len(documents) // BATCH_SIZE + 1}")
    vectorstore.add_documents(batch)
    print(f"Inserted batch {i // BATCH_SIZE + 1} of {len(documents) // BATCH_SIZE + 1}")

print(f"Added {len(documents)} documents to Pinecone")

KeyboardInterrupt: 

# Initializa Groq client

In [ ]:
groq_api_key = userdata.get("GROQ_API_KEY")
os.environ['GROQ_API_KEY'] = groq_api_key

groq_client = Groq(api_key=os.getenv('GROQ_API_KEY'))

# Perform RAG

In [ ]:
# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index(index_name)

In [ ]:
def query(name):
    query = ( f"What are some items that {name} is likely to buy next? "
              "What incentives can I put in place to ensure he or she orders more?"
    )

    return query

In [ ]:
name = "Pirkko Koskitalo"

In [ ]:
query = query(name)
raw_query_embedding = get_huggingface_embeddings(query)

In [ ]:
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=10, include_metadata=True, namespace=namespace)

In [ ]:
contexts = [item['metadata']['text'] for item in top_matches['matches']]

In [ ]:
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [ ]:
print(augmented_query)

<CONTEXT>
Unit Price: 12.0
Total: 540.0
--------------------------------------------------------------------------------------------------
Product: Zaanse koeken
Quantity: 10
Unit Price: 9.5
Total: 95.0
--------------------------------------------------------------------------------------------------
Product: Gnocchi di nonna Alice
Quantity: 45
Unit Price: 38.0
Total: 1710.0
--------------------------------------------------------------------------------------------------
Product: Camembert Pierrot
Quantity: 30
Unit Price: 34.0
Total: 1020.0
Total Price:
Total Price: 4371.6


-------

Purchase Orders
Order ID Order Date Customer Name
11077 2018-05-06 Paula Wilson
Products
Product ID: Product: Quantity: Unit Price:
2 Chang 24 19
3 Aniseed Syrup 4 10
4 Chef Anton's Cajun Seasoning 1 22
6 Grandma's Boysenberry Spread 1 25
7 Uncle Bob's Organic Dried Pears 1 30
8 Northwoods Cranberry Sauce 2 40
10 Ikura 1 31
12 Queso Manchego La Pastora 2 38
13 Konbu 4 6
14 Tofu 1 23.25
16 Pavlova 2 17.45


In [ ]:
system_prompt = f"""You are an expert at understanding and analyzing company data - particularly shipping orders, purchase orders, invoices, and inventory reports.

Answer any questions I have, based on the data provided. Always consider all of the context provided when forming a response.
"""

llm_response = groq_client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content

In [ ]:
print(response)

Based on the provided context, Pirkko Koskitalo has placed orders on several occasions, and the products purchased include:

1. Queso Cabrales (Order ID: 10553)
2. Pavlova (Order ID: 10553)
3. Gustaf's Knäckebröd (Order ID: 10553)
4. Gorgonzola Telino (Order ID: 10553)
5. Steeleye Stout (Order ID: 10553)
6. Chartreuse verte (Order ID: 10455)
7. Perth Pasties (Order ID: 10455)
8. Sirop d'érable (Order ID: 10455)
9. Flotemysost (Order ID: 10455)
10. Chai (Order ID: 10526)
11. Konbu (Order ID: 10526)
12. Gnocchi di nonna Alice (Order ID: 10526)

Analyzing the products purchased, it appears that Pirkko Koskitalo has a preference for:

* Specialty cheeses (Queso Cabrales, Gorgonzola Telino)
* Baked goods (Pavlova)
* Specialty foods (Gustaf's Knäckebröd, Perth Pasties, Sirop d'érable)
* Beverages (Chai, Steeleye Stout)

Considering these preferences, some items that Pirkko Koskitalo might be likely to buy next include:

* Other specialty cheeses (e.g., Camembert Pierrot, Raclette Courdavault

# Putting it all together

In [ ]:
def perform_rag(query):
    raw_query_embedding = get_huggingface_embeddings(query)

    query_embedding = np.array(raw_query_embedding)

    top_matches = pinecone_index.query(vector=query_embedding.tolist(), top_k=10, include_metadata=True, namespace=namespace)

    # Get the list of retrieved texts
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

    # Modify the prompt below as need to improve the response quality
    system_prompt = f"""You are an expert at understanding and analyzing company data - particularly shipping orders, purchase orders, invoices, and inventory reports.

    Answer any questions I have, based on the data provided. Always consider all parts of the context provided when forming a response.
    """

    res = groq_client.chat.completions.create(
        model="llama-3.1-70b-versatile", # llama-3.1-70b-versatile
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    return res.choices[0].message.content

In [ ]:
response = perform_rag(f"What are some trends with {name} purchase orders?")

print(response)

Based on the provided data, here are some trends and observations related to Pirkko Koskitalo's purchase orders:

1. **Frequent Purchases**: Pirkko Koskitalo has placed multiple orders, indicating a regular customer. There are six purchase orders from this customer, with order dates ranging from 2017-02-24 to 2018-04-15.

2. **Diverse Product Selection**: Pirkko Koskitalo's orders comprise a wide range of products, including Chartreuse verte, Perth Pasties, Sirop d'érable, Flotemysost, Queso Cabrales, Pavlova, Gustaf's Knäckebröd, Gorgonzola Telino, Steeleye Stout, Chai, Konbu, Gnocchi di nonna Alice, Tourtière, and Longlife Tofu.

3. **Repeating Products**: Some products appear in multiple orders, such as:
	* Gnocchi di nonna Alice (Order IDs: 10526, 10781)
	* Chai (Order IDs: 10526, 11025)
	* Konbu (Order IDs: 10526, 11025)

This suggests that Pirkko Koskitalo may have a preference for these products or that they are staples in their business.

4. **Increasing Quantities**: For the r